In [88]:
#All needed libraries 
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, silhouette_score 


In [89]:
# Read  data from the data directory
df = pd.read_csv('Fashion_Recommendation_1.csv')

In [90]:
df.sample(5)

,Timestamp,Score,Gender,Age,Skin Color,What kind of clothes do you feel comfortable wearing?,Your Name,What color do you like to wear in Punjabi?,What color do you like to wear in Shirt?,What color do you like to wear in Saree?,What color do you like to wear in Kamiz?,Unnamed: 11,What color clothes do you like to wear?,Email Address
143,NaN,NaN,Male,55+,Black,Shirt,Mohiuddin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2/29/2024 22:15:09,NaN,Female,12-22,Bright,Saree,Sharika Khanom,NaN,NaN,Black,NaN,NaN,NaN,sharikakhanom2512@gmail.com
176,NaN,NaN,Male,41-55,Brown,Shirt,Yasin Arman,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,NaN,NaN,NaN,55+,Bright,Saree,Nobita Chowdhury,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,Male,23-40,Bright,Panjabi,Shafayat Karim,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Data Preprocessing:

In [91]:
#Rename columns and remove unnecessary columns
df.rename(columns={'Timestamp': 'Timestamp', 
                   'Score': 'Score', 
                   'Gender': 'Gender', 
                   'Age': 'Age', 
                   'Skin Color': 'Skin Color',
                   'What kind of clothes do you feel comfortable wearing?': 'Comfortable Clothes', 
                   'Your Name': 'Name', 
                   'What color do you like to wear in Punjabi?': 'Punjabi Color',
                   'What color do you like to wear in Shirt?': 'Shirt Color', 
                   'What color do you like to wear in Saree?': 'Saree Color',
                   'What color do you like to wear in Kamiz?': 'Kamiz Color', 
                   'Unnamed: 11': 'Unused',
                   'What color clothes do you like to wear?': 'Other Preferred Color', 
                   'Email Address': 'Email Address'}, inplace=True)

df.drop(columns=['Unused','Timestamp','Email Address','Score','Name'], inplace=True)

In [92]:
#Remove bangla characters
df = df.apply(lambda x: x.str.split('(', expand=True)[0])

In [93]:
df.columns

Index(['Gender', 'Age', 'Skin Color', 'Comfortable Clothes', 'Punjabi Color',
       'Shirt Color', 'Saree Color', 'Kamiz Color', 'Other Preferred Color'],
      dtype='object')

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Gender                 199 non-null    object
 1   Age                    343 non-null    object
 2   Skin Color             343 non-null    object
 3   Comfortable Clothes    343 non-null    object
 4   Punjabi Color          15 non-null     object
 5   Shirt Color            11 non-null     object
 6   Saree Color            9 non-null      object
 7   Kamiz Color            11 non-null     object
 8   Other Preferred Color  4 non-null      object
dtypes: object(9)
memory usage: 24.2+ KB


In [95]:
# Filter the DataFrame to remove rows where 'Comfortable Clothes' count is less than 5
df = df[df.groupby('Comfortable Clothes')['Comfortable Clothes'].transform('count') >= 5]


In [96]:
#Handel missing value 'Gender' column
df['Gender'] = df['Gender'].str.strip()
df['Gender'].fillna('Female',inplace=True)
df['Gender'].value_counts()


C:\Users\DCL\AppData\Local\Temp\ipykernel_7384\352822309.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Gender'].fillna('Female',inplace=True)


Gender
Male      169
Female    165
Name: count, dtype: int64

In [97]:
# Replace values in the 'Age' column
df['Age'].replace({'12-22': '12-23'}, inplace=True)


C:\Users\DCL\AppData\Local\Temp\ipykernel_7384\3857551778.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].replace({'12-22': '12-23'}, inplace=True)


In [98]:
# Define gender-specific color preferences
male_colors = {'Panjabi': ['Gray', 'Blue', 'White'], 'Shirt': ['Black', 'Blue', 'White']}
female_colors = {'Saree': ['White', 'Blue', 'Black'], 'Kamiz': ['Green', 'Red', 'Black']}

import random

# Define function to fill null values with random colors
def fill_null_color(row):
    colors = []
    if pd.isnull(row['Punjabi Color']):
        if row['Gender'] == 'Male' and row['Comfortable Clothes'] == 'Panjabi':
            colors.append(random.choice(['Gray', 'Blue', 'White']))
        else:
            colors.append('')
    else:
        colors.append(row['Punjabi Color'])
        
    if pd.isnull(row['Shirt Color']):
        if row['Gender'] == 'Male' and row['Comfortable Clothes'] == 'Shirt':
            colors.append(random.choice(['Black', 'Blue', 'White']))
        else:
            colors.append('')
    else:
        colors.append(row['Shirt Color']) 
        
    if pd.isnull(row['Saree Color']):
        if row['Gender'] == 'Female' and row['Comfortable Clothes'] == 'Saree':
            colors.append(random.choice(['White', 'Blue', 'Black']))
        else:
            colors.append('')
    else:
        colors.append(row['Saree Color'])
        
    if pd.isnull(row['Kamiz Color']):
        if row['Gender'] == 'Female' and row['Comfortable Clothes'] == 'Salwar Kamiz':
            colors.append(random.choice(['Green', 'Red', 'Black']))
        else:
            colors.append('')
    else:
        colors.append(row['Kamiz Color'])
    
    return pd.Series(colors, index=['Punjabi Color', 'Shirt Color', 'Saree Color', 'Kamiz Color'])


df[['Punjabi Color', 'Shirt Color', 'Saree Color', 'Kamiz Color']] = df.apply(fill_null_color, axis=1)


In [99]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 334 entries, 5 to 342
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Gender                 334 non-null    object
 1   Age                    334 non-null    object
 2   Skin Color             334 non-null    object
 3   Comfortable Clothes    334 non-null    object
 4   Punjabi Color          334 non-null    object
 5   Shirt Color            334 non-null    object
 6   Saree Color            334 non-null    object
 7   Kamiz Color            334 non-null    object
 8   Other Preferred Color  0 non-null      object
dtypes: object(9)
memory usage: 26.1+ KB


#### Data Training and Testing:

In [100]:

X = df[['Gender', 'Age', 'Skin Color']]
y_clothes = df['Comfortable Clothes']
y_color = df[['Punjabi Color', 'Shirt Color', 'Saree Color', 'Kamiz Color', 'Other Preferred Color']]

X_train, X_test, y_clothes_train, y_clothes_test, y_color_train, y_color_test = train_test_split(X, y_clothes, y_color, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()

y_clothes_train_encoded = label_encoder.fit_transform(y_clothes_train)
y_clothes_test_encoded = label_encoder.transform(y_clothes_test)


encoder = ce.TargetEncoder(cols=['Gender', 'Age', 'Skin Color'])

X_train_encoded = encoder.fit_transform(X_train, y_clothes_train_encoded)

X_test_encoded = encoder.transform(X_test)



#### Apply Mechanic Learning Model 

In [101]:
# Initialize RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train_encoded, y_clothes_train_encoded)

y_clothes_pred = clf.predict(X_test_encoded)

# Initialize and train KNN classifier
knn = KNeighborsClassifier()
knn.fit(X_train_encoded, y_clothes_train)

dt = DecisionTreeClassifier()
dt.fit(X_train_encoded, y_clothes_train)

# Initialize and fit K-Means Clustering
kmeans = KMeans(n_clusters=5)  
kmeans.fit(X_train_encoded)

y_clothes_pred_knn = knn.predict(X_test_encoded)

y_clothes_pred_dt = dt.predict(X_test_encoded)

cluster_pred = kmeans.predict(X_test_encoded)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_clothes_test, y_clothes_pred_knn))

# Evaluate the model
print("Classification Report for Comfortable Clothes Prediction:")
print(classification_report(y_clothes_test_encoded, y_clothes_pred))

# Evaluate Decision Tree
print("Decision Tree Classification Report:")
print(classification_report(y_clothes_test, y_clothes_pred_dt))

# Analyze K-Means Clusters
print("K-Means Clusters:")
print(cluster_pred)


KNN Classification Report:
              precision    recall  f1-score   support

     Panjabi       0.38      0.62      0.48        16
Salwar Kamiz       0.53      0.57      0.55        14
       Saree       0.54      0.50      0.52        14
       Shirt       0.54      0.30      0.39        23

    accuracy                           0.48        67
   macro avg       0.50      0.50      0.48        67
weighted avg       0.50      0.48      0.47        67

Classification Report for Comfortable Clothes Prediction:
              precision    recall  f1-score   support

           0       0.38      0.69      0.49        16
           1       0.38      0.36      0.37        14
           2       0.40      0.43      0.41        14
           3       0.50      0.22      0.30        23

    accuracy                           0.40        67
   macro avg       0.42      0.42      0.39        67
weighted avg       0.43      0.40      0.38        67

Decision Tree Classification Report:
        

#### Model Accuracy

In [102]:

classifiers = {
    "Random Forest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "K-Means Clustering": KMeans(n_clusters=2) 
}

accuracies = []
labels = []

for label, clf in classifiers.items():
    if label != "K-Means Clustering":
        clf.fit(X_train_encoded, y_clothes_train_encoded)
        y_pred = clf.predict(X_test_encoded)
        accuracy = accuracy_score(y_clothes_test_encoded, y_pred)
        accuracies.append(accuracy)
        labels.append(label)
    else:
        clf.fit(X_train_encoded)
        score = silhouette_score(X_train_encoded, clf.labels_)
        accuracies.append(score)
        labels.append(label + " Silhouette Score")

#Check accuracy plot
plt.figure(figsize=(5, 3))
plt.barh(labels, accuracies, color='skyblue')
plt.xlabel('Accuracy / Silhouette Score')
plt.title('Accuracy of Different Algorithms')
plt.xlim(0, 1)  
plt.show()


C:\Users\DCL\AppData\Local\Temp\ipykernel_7384\2356442030.py:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


#### Here we can KNN give best accuracy ,so that we use next prediction using KNN

In [103]:

# Define features and target variables
X = df[['Gender', 'Age', 'Skin Color']].copy()  
y_clothes = df['Comfortable Clothes']
y_color = df[['Punjabi Color', 'Shirt Color', 'Saree Color', 'Kamiz Color']]

label_encoder_input = LabelEncoder()
X['Gender'] = label_encoder_input.fit_transform(X['Gender'])

X['Age'] = pd.to_numeric(X['Age'], errors='coerce')

age_bins = [0, 22, 40, 55, float('inf')]
age_labels = ['12-22', '23-40', '41-55', '55+']

X['Age'] = pd.cut(X['Age'], bins=age_bins, labels=age_labels, right=False)

label_encoder_age = LabelEncoder()
X['Age'] = label_encoder_age.fit_transform(X['Age'])

label_encoder_skin_color = LabelEncoder()
X['Skin Color'] = label_encoder_skin_color.fit_transform(X['Skin Color'])

label_encoder_clothes = LabelEncoder()
y_clothes_encoded = label_encoder_clothes.fit_transform(y_clothes)

X_train_clothes, X_test_clothes, y_clothes_train, y_clothes_test = train_test_split(X, y_clothes_encoded, test_size=0.2, random_state=42)

knn_classifier_clothes = KNeighborsClassifier()
knn_classifier_clothes.fit(X_train_clothes, y_clothes_train)


KNeighborsClassifier()

In [104]:
# User input for prediction
user_input = {
    'Gender': 'Female',  
    'Age': '12-23',  
    'Skin Color': 'Brown'  
}


In [105]:
def Punjabi_Color(user_input, df):

    Punjabi_df = df[df['Comfortable Clothes'] == 'Punjabi']
    X = Punjabi_df[['Gender', 'Age', 'Skin Color']]
    y_color = Punjabi_df['Punjabi Color']

    label_encoders = {}
    for column in ['Gender', 'Age', 'Skin Color']:
        label_encoders[column] = LabelEncoder()
        X[column] = label_encoders[column].fit_transform(X[column])

    label_encoder_color = LabelEncoder()
    y_color_encoded = label_encoder_color.fit_transform(y_color)

    knn_classifier_color = KNeighborsClassifier()
    knn_classifier_color.fit(X, y_color_encoded)

    input_df = pd.DataFrame(user_input, index=[0])

    for column in ['Gender', 'Age', 'Skin Color']:
        input_df[column] = label_encoders[column].transform([user_input[column]])

    color_prediction = knn_classifier_color.predict(input_df)

    predicted_color = label_encoder_color.inverse_transform(color_prediction)[0]

    print("Predicted Punjabi Color:", predicted_color)




In [106]:
def Shirt_Color(user_input, df):

        Shirt_df = df[df['Comfortable Clothes'] == 'Shirt']
        X = Shirt_df[['Gender', 'Age', 'Skin Color']]
        y_color = Shirt_df['Shirt Color']

        label_encoders = {}
        for column in ['Gender', 'Age', 'Skin Color']:
            label_encoders[column] = LabelEncoder()
            X[column] = label_encoders[column].fit_transform(X[column])

        label_encoder_color = LabelEncoder()
        y_color_encoded = label_encoder_color.fit_transform(y_color)

        knn_classifier_color = KNeighborsClassifier()
        knn_classifier_color.fit(X, y_color_encoded)

        input_df = pd.DataFrame(user_input, index=[0])

        for column in ['Gender', 'Age', 'Skin Color']:
            input_df[column] = label_encoders[column].transform([user_input[column]])

        color_prediction = knn_classifier_color.predict(input_df)
        predicted_color = label_encoder_color.inverse_transform(color_prediction)[0]

        print("Predicted Shirt Color:", predicted_color)


In [107]:
def Kamiz_Color(user_input, df):

        kamiz_df = df[df['Comfortable Clothes'] == 'Salwar Kamiz']
        X = kamiz_df[['Gender', 'Age', 'Skin Color']]
        y_color = kamiz_df['Kamiz Color']

        label_encoders = {}
        for column in ['Gender', 'Age', 'Skin Color']:
            label_encoders[column] = LabelEncoder()
            X[column] = label_encoders[column].fit_transform(X[column])

        label_encoder_color = LabelEncoder()
        y_color_encoded = label_encoder_color.fit_transform(y_color)

        knn_classifier_color = KNeighborsClassifier()
        knn_classifier_color.fit(X, y_color_encoded)
        input_df = pd.DataFrame(user_input, index=[0])

        for column in ['Gender', 'Age', 'Skin Color']:
            input_df[column] = label_encoders[column].transform([user_input[column]])

        color_prediction = knn_classifier_color.predict(input_df)

        predicted_color = label_encoder_color.inverse_transform(color_prediction)[0]

        print("Predicted Salwar Kamiz Color:", predicted_color)


In [108]:
def Saree_Color(user_input, df):

        saree_df = df[df['Comfortable Clothes'] == 'Saree']
        X = saree_df[['Gender', 'Age', 'Skin Color']]
        y_color = saree_df['Saree Color']

        label_encoders = {}
        for column in ['Gender', 'Age', 'Skin Color']:
            label_encoders[column] = LabelEncoder()
            X[column] = label_encoders[column].fit_transform(X[column])
        label_encoder_color = LabelEncoder()
        y_color_encoded = label_encoder_color.fit_transform(y_color)

        knn_classifier_color = KNeighborsClassifier()
        knn_classifier_color.fit(X, y_color_encoded)
        input_df = pd.DataFrame(user_input, index=[0])

        for column in ['Gender', 'Age', 'Skin Color']:
            input_df[column] = label_encoders[column].transform([user_input[column]])

        color_prediction = knn_classifier_color.predict(input_df)
        predicted_color = label_encoder_color.inverse_transform(color_prediction)[0]

        print("Predicted Saree Color:", predicted_color)


In [109]:
# Predict comfortable clothes

X = df[['Gender', 'Age', 'Skin Color']]
y_clothes = df['Comfortable Clothes']

label_encoders = {}
for column in ['Gender', 'Age', 'Skin Color']:
    label_encoders[column] = LabelEncoder()
    X[column] = label_encoders[column].fit_transform(X[column])

label_encoder_clothes = LabelEncoder()
y_clothes_encoded = label_encoder_clothes.fit_transform(y_clothes)

knn_classifier_clothes = KNeighborsClassifier()
knn_classifier_clothes.fit(X, y_clothes_encoded)

input_df = pd.DataFrame(user_input, index=[0])

for column in ['Gender', 'Age', 'Skin Color']:
    input_df[column] = label_encoders[column].transform([user_input[column]])


clothes_prediction = knn_classifier_clothes.predict(input_df)
predicted_cloth = label_encoder_clothes.inverse_transform(clothes_prediction)[0]


C:\Users\DCL\AppData\Local\Temp\ipykernel_7384\397568202.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoders[column].fit_transform(X[column])
C:\Users\DCL\AppData\Local\Temp\ipykernel_7384\397568202.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoders[column].fit_transform(X[column])
C:\Users\DCL\AppData\Local\Temp\ipykernel_7384\397568202.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [110]:
print("Predicted Comfortable Clothes:", predicted_cloth)  
# Call different functions based on the predicted comfortable clothes
if predicted_cloth == 'Panjabi':
    predicted_color = Punjabi_Color(user_input, df)
elif predicted_cloth == 'Shirt':
    predicted_color = Shirt_Color(user_input, df)
elif predicted_cloth == 'Saree':
    predicted_color = Saree_Color(user_input, df)
elif predicted_cloth == 'Salwar Kamiz':
    predicted_color = Kamiz_Color(user_input, df)
else:
    print("No color prediction function available for the predicted comfortable clothes.")

 

Predicted Comfortable Clothes: Saree
Predicted Saree Color: Black


C:\Users\DCL\AppData\Local\Temp\ipykernel_7384\3300356957.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoders[column].fit_transform(X[column])
C:\Users\DCL\AppData\Local\Temp\ipykernel_7384\3300356957.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoders[column].fit_transform(X[column])
C:\Users\DCL\AppData\Local\Temp\ipykernel_7384\3300356957.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo